# Tests For Different Types of Models

## Functional API

### Structure
#### Inputs:
- Title (List of existance of words)
- Text body (list of existance of words)
- Tags (one-hot encoded)

#### Outputs
- Priority score (0-1)
- Department (softmax)

In [1]:
# Imports
import numpy as np
import tensorflow as tf
import tensorflow.keras as keras
import tensorflow.keras.layers as layers

c:\users\danie\pycharmprojects\tensorflowtest\venv\lib\site-packages\numpy\_distributor_init.py:30: UserWarning: loaded more than 1 DLL from .libs:
c:\users\danie\pycharmprojects\tensorflowtest\venv\lib\site-packages\numpy\.libs\libopenblas.GK7GX5KEQ4F6UYO3P26ULGBQYHGQO7J4.gfortran-win_amd64.dll
c:\users\danie\pycharmprojects\tensorflowtest\venv\lib\site-packages\numpy\.libs\libopenblas.WCDJNK7YVMPZQ2ME2ZZHJJRJ3JIKNDB7.gfortran-win_amd64.dll
  warnings.warn("loaded more than 1 DLL from .libs:"


In [7]:
# declare constants
vocab_size = 10000
num_tags = 100
num_depart = 4

# the 3 types of inputs
title = keras.Input(shape=(vocab_size,), name="title")
text_body = keras.Input(shape=(vocab_size,), name="text_body")
tags = keras.Input(shape=(num_tags,), name="tags")

# Processing Data
features = layers.Concatenate()([title, text_body, tags])
features = layers.Dense(64, activation="relu", name="features")(features)

# the 2 types of outputs
priority = layers.Dense(1, activation="sigmoid", name="priority")(features)
deparment = layers.Dense(num_depart, activation="softmax", name="department")(features)

model = keras.Model(inputs=[title, text_body, tags],
                    outputs=[priority, deparment])

In [3]:
keras.utils.plot_model(model, "ticket_classifier.png")

# take a look at the model
model.summary()

('You must install pydot (`pip install pydot`) and install graphviz (see instructions at https://graphviz.gitlab.io/download/) ', 'for plot_model/model_to_dot to work.')
Model: "model"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
title (InputLayer)              [(None, 10000)]      0                                            
__________________________________________________________________________________________________
text_body (InputLayer)          [(None, 10000)]      0                                            
__________________________________________________________________________________________________
tags (InputLayer)               [(None, 100)]        0                                            
__________________________________________________________________________________________________
concatenate (Concatenat

In [4]:
# training

# declare data points
num_samples = 1280
title_data = np.random.randint(0, 2, size=(num_samples, vocab_size))
body_data = np.random.randint(0, 2, size=(num_samples, vocab_size))
tag_data = np.random.randint(0, 2, size=(num_samples, num_tags))

priority_data = np.random.random(size=(num_samples, 1))
# sparse categorical data
department_data = np.random.randint(0, num_depart, size=(num_samples,))

In [ ]:
# note different loss & metrics for each output
model.compile(optimizer="rmsprop",
              loss=["mean_squared_error", "sparse_categorical_crossentropy"],
              metrics=[["mean_absolute_error"], ["accuracy"]])

# fit model
history = model.fit(
    [title_data, body_data, tag_data],
    [priority_data, department_data],
    epochs=8
)

print(model.evaluate(
    [title_data, body_data, tag_data],
    [priority_data, department_data]
))

priority_pred, department_pred = model.predict([title_data, body_data, tag_data])


In [8]:
# fit model, rely on layer names instead of input order
# DO NOTE THAT THIS CAN'T BE DONE WITH THE PREVIOUS CODE BLOCK
model.compile(
    optimizer="rmsprop",
    loss={"priority": "mean_squared_error", "department": "sparse_categorical_crossentropy"},
    metrics={"priority": ["mean_absolute_error"], "department": ["accuracy"]}
)

# add a callback and store logs with tensorboard
model.fit(
    {"title": title_data, "text_body": body_data, "tags": tag_data},
    {"priority": priority_data, "department": department_data},
    epochs=8,
    callbacks=[keras.callbacks.TensorBoard("./logs")]
)

print(model.evaluate(
    {"title": title_data, "text_body": body_data, "tags": tag_data},
    {"priority": priority_data, "department": department_data},
))

priority_pred_2, department_pred_2 = model.predict(
    {"title": title_data, "text_body": body_data, "tags": tag_data}
)

Epoch 1/8
40/40 [==============================] - 1s 14ms/step - loss: 5.2999 - priority_loss: 0.2239 - department_loss: 5.0760 - priority_mean_absolute_error: 0.3929 - department_accuracy: 0.2414

In [41]:
# adding a new layer
# note that features = model.layers[4].output
difficulty = layers.Dense(3, activation="softmax", name="difficulty")(model.layers[4].output)
new_model = keras.Model(inputs=[title, text_body, tags], outputs=[priority, deparment, difficulty])

new_model.summary()



Model: "model_5"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
title (InputLayer)              [(None, 10000)]      0                                            
__________________________________________________________________________________________________
text_body (InputLayer)          [(None, 10000)]      0                                            
__________________________________________________________________________________________________
tags (InputLayer)               [(None, 100)]        0                                            
__________________________________________________________________________________________________
concatenate_2 (Concatenate)     (None, 20100)        0           title[0][0]                      
                                                                 text_body[0][0]            